In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
df_events = pd.read_csv('./vi_data/events_train.csv')
df_purchases = pd.read_csv('./vi_data/purchases_train.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Nápady

https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea
https://www.youtube.com/watch?v=Ty3Ui_mQtxY

# Prehľad dát

In [3]:
df_events.head()

,customer_id,timestamp,event_type,product_id,title,category_name,price
0,204685f9-00f0-4a36-bab1-e0b74673c3b1,2019-07-01 00:00:01.101977 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,NaN,747.49
1,552445a5-f41d-4b25-b479-4e776d9bee61,2019-07-01 00:00:01.901589 UTC,view_item,29db8f57-49bd-4108-a0bb-0d245278e999,Puma Woven Shorts Girls,NaN,11.92
2,fac01f48-a03a-41f2-acd5-b5f4e407d9c2,2019-07-01 00:00:08.954928 UTC,view_item,a90bddc4-8eb7-4c99-aa4f-2f2377c246a2,Trendyol Multi-colored Flywheel Detailed Swimwear,NaN,23.85
3,552445a5-f41d-4b25-b479-4e776d9bee61,2019-07-01 00:00:09.721589 UTC,view_item,600268e1-a2ba-4b1b-ab57-58c577412af0,Puma Woven Shorts Girls,NaN,11.92
4,bc59a341-040f-41ea-8f33-cc4a02c2d85d,2019-07-01 00:00:13.745257 UTC,view_item,b74e0b38-1efe-4d86-9718-952cc2580212,Women's dress ROXY NEW LEASEOFLIFE J KTDR,NaN,40.00


In [4]:
df_purchases.head()

,customer_id,timestamp,event_type,product_id,title,category_name,price
0,0d1b7397-7d3c-44c0-9efc-d38bf197828b,2019-07-01 00:05:54.308966 UTC,purchase_item,954f9f2c-d3ca-4236-ac9a-4ea7bcf09305,Trendyol Leopard print Bikini bottom,NaN,NaN
1,81ccae7e-e496-4997-a289-4669bf53f33e,2019-07-01 00:20:03.404186 UTC,purchase_item,75b281e5-8a16-42cb-9ae0-9a98db7a2c40,No Fear Reflect Shorts Mens,NaN,NaN
2,50777c55-8dd6-4309-a5ca-26e66c8a8279,2019-07-01 00:34:35.989935 UTC,purchase_item,0112dec8-47f5-4c2c-9109-571e2dbb6345,Trendyol Multicolor camisole Blouse,NaN,NaN
3,50777c55-8dd6-4309-a5ca-26e66c8a8279,2019-07-01 00:34:35.991935 UTC,purchase_item,0fa25a2d-2aa1-4397-82f1-5a64f3b1272d,Trendyol White High Waist Skinny Jeans,NaN,NaN
4,647d269f-b18f-4558-8653-93369d862ec9,2019-07-01 00:52:53.083698 UTC,purchase_item,81c01216-55a9-4588-a722-bccf0bf35fd5,Firetrap Lazer Polo Shirt,NaN,NaN


## Informácie o dátach

In [5]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14614385 entries, 0 to 14614384
Data columns (total 7 columns):
customer_id      object
timestamp        object
event_type       object
product_id       object
title            object
category_name    object
price            float64
dtypes: float64(1), object(6)
memory usage: 780.5+ MB


In [6]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188712 entries, 0 to 188711
Data columns (total 7 columns):
customer_id      188712 non-null object
timestamp        188712 non-null object
event_type       188712 non-null object
product_id       188712 non-null object
title            188684 non-null object
category_name    143814 non-null object
price            143826 non-null float64
dtypes: float64(1), object(6)
memory usage: 10.1+ MB


# Predspracovanie dát

In [7]:
from sklearn import preprocessing

In [8]:
# spojenie dát
df = pd.concat([df_events, df_purchases])

del [[df_events, df_purchases]]
gc.collect()

0

In [10]:
df.sort_values(by=['timestamp'], inplace=True, ascending=True)

In [11]:
df.drop(['price'], axis=1, inplace=True)

In [12]:
df['event_type'] = df['event_type'].replace({ 'view_item': 1, 'purchase_item': 2 })

## Zakódovanie idčiek

In [13]:
from sklearn import preprocessing

In [14]:
le_customer = preprocessing.LabelEncoder()
le_product = preprocessing.LabelEncoder()

df['customer_id'] = le_customer.fit_transform(df['customer_id'])
df['product_id'] = le_product.fit_transform(df['product_id'])

## Rozdelenie na trénovaciu a testovaciu zložku

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
df_train, df_test = train_test_split(df, test_size=0.20, shuffle=False)

In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11842477 entries, 0 to 11689792
Data columns (total 6 columns):
customer_id      int64
timestamp        object
event_type       int64
product_id       int64
title            object
category_name    object
dtypes: int64(3), object(3)
memory usage: 632.5+ MB


In [18]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2960620 entries, 11689793 to 14614384
Data columns (total 6 columns):
customer_id      int64
timestamp        object
event_type       int64
product_id       int64
title            object
category_name    object
dtypes: int64(3), object(3)
memory usage: 158.1+ MB


### Prehľad trénovacích a testovacích dát

In [20]:
df_test.head()

,customer_id,timestamp,event_type,product_id,title,category_name
11689793,1218557,2019-08-21 07:57:53.361388 UTC,1,26125,Trendyol Multi-colored Plisse Skirt,Women>Women's Clothing>Skirts>Midi skirts
11689794,545607,2019-08-21 07:57:53.372592 UTC,1,54241,"Trendyol He's The Only Color We Block, Swimsuit",Women>Women's Clothing>Swimmsuits & bikins>One...
11689795,881435,2019-08-21 07:57:53.532665 UTC,1,54376,Women's jacket NORTHFINDER EMMALINE,Women>Women's Clothing>Jackets & Coats>Quilted...
11689796,262051,2019-08-21 07:57:53.700887 UTC,1,27195,Men's t-shirt KILPI AREC-M,Men>Men Clothing>T-Shirts & Polo shirts>Short ...
11689797,2064097,2019-08-21 07:57:53.86051 UTC,1,104807,Made Of Emotion Woman's Dress M417 Navy Blue,Women>Women's Clothing>Dresses>Mini dress


In [21]:
df_train.head()

,customer_id,timestamp,event_type,product_id,title,category_name
0,279709,2019-07-01 00:00:01.101977 UTC,1,116557,Michael Kors Nappa Leather Racer Jacket,NaN
1,735927,2019-07-01 00:00:01.901589 UTC,1,20221,Puma Woven Shorts Girls,NaN
2,2166327,2019-07-01 00:00:08.954928 UTC,1,81561,Trendyol Multi-colored Flywheel Detailed Swimwear,NaN
3,735927,2019-07-01 00:00:09.721589 UTC,1,46573,Puma Woven Shorts Girls,NaN
4,1628326,2019-07-01 00:00:13.745257 UTC,1,88433,Women's dress ROXY NEW LEASEOFLIFE J KTDR,NaN


# Tvorba sparse matrix

robím podla tohoto
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea

Ktorú sparse_matrix použiť:
https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/

In [22]:
from scipy.sparse import coo_matrix

In [23]:
# tvorba sparse matrix
mat_coo = coo_matrix(
    (
        np.array(df_train['event_type']), 
        (
            np.array(df_train['product_id']), 
            np.array(df_train['customer_id'])
        )
    )
)

In [24]:
mat_csr = mat_coo.tocsr(copy=False)

In [25]:
mat_csr.shape

(123837, 2211386)

# Odporúčanie

In [26]:
import operator
from sklearn.neighbors import NearestNeighbors

In [27]:
# vytvorenie inštancie KNN
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

# natrénovanie na trénovacích dátach
model_knn.fit(mat_csr)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [60]:
# funkcia, ktorá vráti odporúčania pre daného customera
def recommend(model_knn, customer_id, n_recos): 
    customer_id_inversed = le_customer.inverse_transform([customer_id])
    
    # 1. Nájdi všetky produkty, ktoré si zákazník pozrel/kúpil v train
    interacted_products_train = df_train[df_train['customer_id'] == customer_id]['product_id'].values
    
    # 2. Ak nema ziadne produkty v train, tak daj generické odporúčania a skonči
    if len(interacted_products_train) == 0:
        return [], [], 0
        # TODO

        
    # 3. Generovanie odporúčaní pomocou KNN
    # n_recos+1, pretože vždy vráti aj seba
    distances2D, indices2D = model_knn.kneighbors(mat_csr[interacted_products_train], n_neighbors=n_recos+1)
    
    
    # 4. Vytvorenie mapy s najmenšími vzdialenostami pre každý produkt
    index_distance_map = {}
    for distances, indices in zip(distances2D, indices2D):
        for distance, index in zip(distances[1:], indices[1:]): 
            if index in index_distance_map and index_distance_map[index] < distance:
                continue
            index_distance_map[index] = distance
            

    # 5. nájdi najlepších n (n_recos) výsledkov
    indices = []
    distances = []
    for _ in range(n_recos):
        index = min(index_distance_map.items(), key=operator.itemgetter(1))[0]
        indices.append(index)
        distances.append(index_distance_map[index])
        del index_distance_map[index]
        

    interacted_products_test = df_test[df_test['customer_id'] == customer_id]['product_id'].values
        
    # 6. Výpočet precision
    precision = calc_precision(
        indices,
        list(dict.fromkeys(interacted_products_test)) # odstrániť duplikáty z pola produktov pozretých v testovacej sade
    )
    
    return distances, indices, precision

In [64]:
def calc_precision(predicted_item_ids, future_touched_item_ids):
    tp = intersection(predicted_item_ids, future_touched_item_ids)
    return (len(tp) / len(predicted_item_ids))


def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3  

In [74]:
# Náhodný výber customera z testovacej množiny
import random
random_customer = random.choice(df_test['customer_id'].values)

# všetci customeri z testovacej množiny
customers = df_test['customer_id'].values

overall_precision = 0
print(f'Number of customers: {len(customers)}')
for idx in range(10000):
    distances, indices, precision = recommend(model_knn, customers[idx], 10)
    
    overall_precision = overall_precision + precision
    
    if precision > 0:
        print(f'Cust. {idx + 1}.\t{customers[idx]}\t{precision * 100}%\t{indices}')
        
print(f'Overall precision is {overall_precision / 10000}')

Number of customers: 2960620
Cust. 4.	262051	20.0%	[115280, 62752, 118264, 118730, 80729, 121145, 38025, 18235, 19189, 67284]
Cust. 6.	1561885	20.0%	[113838, 114025, 83089, 13499, 13798, 35427, 80883, 113682, 92870, 51588]
Cust. 7.	335878	20.0%	[42155, 25166, 79361, 23112, 11878, 82603, 55085, 119901, 31178, 57081]
Cust. 9.	161403	10.0%	[89995, 6899, 89523, 22150, 4145, 76954, 47608, 38973, 42903, 38974]
Cust. 14.	161403	10.0%	[89995, 6899, 89523, 22150, 4145, 76954, 47608, 38973, 42903, 38974]
Cust. 17.	922821	20.0%	[49892, 86941, 18721, 93715, 83228, 92605, 15100, 104186, 8342, 100397]
Cust. 18.	1342851	30.0%	[71069, 111024, 18887, 79637, 57860, 33238, 97500, 7706, 86243, 72125]
Cust. 19.	161403	10.0%	[89995, 6899, 89523, 22150, 4145, 76954, 47608, 38973, 42903, 38974]
Cust. 23.	1274232	10.0%	[111816, 71601, 29537, 91624, 93532, 115631, 88432, 89428, 26273, 30501]
Cust. 28.	1561885	20.0%	[113838, 114025, 83089, 13499, 13798, 35427, 80883, 113682, 92870, 51588]
Cust. 29.	151689	10.0%	

Cust. 244.	475415	10.0%	[85108, 20290, 99506, 108094, 20221, 69086, 46573, 93982, 73946, 49618]
Cust. 245.	1607497	10.0%	[68735, 91820, 67011, 52470, 99479, 113451, 72173, 78613, 84851, 25425]
Cust. 247.	1857736	10.0%	[102520, 7376, 40308, 2895, 115893, 120276, 61664, 83460, 63208, 103892]
Cust. 250.	611391	20.0%	[55344, 15054, 111074, 57134, 11938, 21832, 54376, 52914, 31053, 95119]
Cust. 258.	1977149	10.0%	[54917, 78113, 62003, 58694, 58193, 41, 118180, 102285, 13737, 100885]
Cust. 260.	499225	10.0%	[5644, 18763, 101432, 30594, 84553, 103283, 4233, 59166, 89874, 98051]
Cust. 262.	1607497	10.0%	[68735, 91820, 67011, 52470, 99479, 113451, 72173, 78613, 84851, 25425]
Cust. 263.	621552	20.0%	[123493, 20496, 106525, 82457, 87164, 80753, 18429, 38304, 101987, 49159]
Cust. 264.	1154976	30.0%	[86243, 30124, 18887, 79637, 70468, 121848, 65820, 84025, 75776, 19753]
Cust. 266.	1139204	20.0%	[13055, 55877, 96583, 62759, 27137, 8167, 119036, 30262, 58013, 37670]
Cust. 267.	1979488	10.0%	[1099, 45

Cust. 597.	1152730	10.0%	[110495, 58112, 32845, 90915, 121145, 116660, 6620, 6101, 80999, 109584]
Cust. 606.	723089	10.0%	[103855, 59487, 100927, 90258, 55877, 9830, 36784, 84132, 32310, 86243]
Cust. 607.	2002278	10.0%	[106721, 108551, 80777, 50259, 59735, 117906, 108742, 97031, 8638, 40271]
Cust. 613.	1645698	10.0%	[110062, 43679, 31473, 123431, 15406, 86335, 104842, 11535, 103496, 68309]
Cust. 615.	2087002	10.0%	[78182, 22290, 3031, 51541, 76248, 18136, 50658, 11912, 36222, 9760]
Cust. 617.	1152730	10.0%	[110495, 58112, 32845, 90915, 121145, 116660, 6620, 6101, 80999, 109584]
Cust. 618.	1380100	40.0%	[66069, 77697, 86493, 69302, 81674, 84077, 6701, 64356, 45652, 8598]
Cust. 619.	611391	20.0%	[55344, 15054, 111074, 57134, 11938, 21832, 54376, 52914, 31053, 95119]
Cust. 626.	2056384	20.0%	[38136, 11417, 75343, 101345, 66668, 106317, 111246, 95480, 82093, 118427]
Cust. 633.	2087002	10.0%	[78182, 22290, 3031, 51541, 76248, 18136, 50658, 11912, 36222, 9760]
Cust. 634.	262051	20.0%	[115280

Cust. 891.	1086054	10.0%	[112093, 105046, 108257, 103217, 65395, 20035, 94659, 97930, 37517, 43096]
Cust. 895.	2061625	10.0%	[40102, 69593, 87630, 107340, 11132, 103930, 19078, 7871, 11460, 24947]
Cust. 897.	1086054	10.0%	[112093, 105046, 108257, 103217, 65395, 20035, 94659, 97930, 37517, 43096]
Cust. 900.	1048495	30.0%	[55495, 72805, 83249, 68867, 96819, 29601, 56774, 8475, 22271, 100737]
Cust. 904.	1086054	10.0%	[112093, 105046, 108257, 103217, 65395, 20035, 94659, 97930, 37517, 43096]
Cust. 908.	1796894	20.0%	[21057, 37538, 68790, 75539, 81107, 28241, 1920, 90572, 49192, 111296]
Cust. 915.	1796894	20.0%	[21057, 37538, 68790, 75539, 81107, 28241, 1920, 90572, 49192, 111296]
Cust. 917.	1645698	10.0%	[110062, 43679, 31473, 123431, 15406, 86335, 104842, 11535, 103496, 68309]
Cust. 918.	426593	10.0%	[110499, 116676, 64969, 45308, 52428, 72123, 11633, 87659, 80590, 18436]
Cust. 919.	1172473	10.0%	[89743, 75204, 34016, 26683, 33560, 66407, 23082, 73449, 5763, 19103]
Cust. 922.	101478	30.0%

Cust. 1197.	907408	10.0%	[34136, 50689, 113916, 29506, 63891, 123618, 23444, 30759, 95237, 41214]
Cust. 1199.	1782578	20.0%	[20496, 123493, 106525, 83089, 113838, 50868, 96120, 114172, 51229, 114025]
Cust. 1201.	1009740	10.0%	[111163, 23460, 85990, 118320, 56289, 97463, 50175, 96578, 108296, 67627]
Cust. 1206.	908279	30.0%	[8360, 10664, 64458, 61231, 109653, 34136, 29673, 50689, 20059, 49400]
Cust. 1213.	1481026	50.0%	[107239, 71663, 95239, 97619, 13092, 102852, 42814, 106846, 36138, 11024]
Cust. 1215.	476301	30.0%	[31242, 74348, 46584, 54408, 104099, 112205, 101491, 27538, 108421, 77659]
Cust. 1225.	254363	30.0%	[67976, 79353, 37813, 82609, 118850, 71346, 111151, 37274, 79318, 30215]
Cust. 1228.	254363	30.0%	[67976, 79353, 37813, 82609, 118850, 71346, 111151, 37274, 79318, 30215]
Cust. 1229.	1782578	20.0%	[20496, 123493, 106525, 83089, 113838, 50868, 96120, 114172, 51229, 114025]
Cust. 1232.	605550	10.0%	[110495, 45308, 106585, 52428, 89381, 32845, 80475, 120317, 52622, 10806]
Cust. 1

...

Cust. 8480.	183917	20.0%	[82372, 78198, 10922, 79291, 71771, 62717, 30870, 49608, 13107, 113079]
Cust. 8482.	431393	20.0%	[1444, 94301, 218, 117649, 30739, 123034, 14478, 107964, 17546, 18883]
Cust. 8486.	1475211	20.0%	[79637, 75776, 45542, 65820, 44797, 32310, 121848, 86243, 7447, 1290]
Cust. 8488.	183917	20.0%	[82372, 78198, 10922, 79291, 71771, 62717, 30870, 49608, 13107, 113079]
Cust. 8490.	164541	40.0%	[18208, 48395, 30225, 85354, 14018, 38033, 100238, 101572, 112954, 62648]
Cust. 8500.	183917	20.0%	[82372, 78198, 10922, 79291, 71771, 62717, 30870, 49608, 13107, 113079]
Cust. 8503.	2025046	20.0%	[71141, 91764, 45641, 115147, 101725, 14919, 72123, 11633, 87659, 80590]
Cust. 8512.	664973	10.0%	[116660, 7521, 55936, 16078, 63388, 8687, 28456, 40791, 23146, 93195]
Cust. 8528.	958728	20.0%	[102242, 4306, 106568, 57588, 63914, 85026, 121599, 53745, 40075, 90442]
Cust. 8538.	1119219	30.0%	[35413, 20007, 68222, 53240, 93945, 101610, 78960, 84401, 95276, 92968]
Cust. 8541.	1941438	20.0%	[7

Cust. 8906.	101478	30.0%	[106888, 64163, 107239, 24096, 71663, 3403, 95239, 97619, 120974, 9460]
Cust. 8907.	1383957	10.0%	[99523, 10386, 24901, 61195, 110090, 63855, 60180, 123579, 39918, 62040]
Cust. 8930.	431393	20.0%	[1444, 94301, 218, 117649, 30739, 123034, 14478, 107964, 17546, 18883]
Cust. 8939.	2124299	40.0%	[69143, 73482, 122355, 16686, 37721, 25127, 45958, 66252, 99710, 79090]
Cust. 8944.	1796894	20.0%	[21057, 37538, 68790, 75539, 81107, 28241, 1920, 90572, 49192, 111296]
Cust. 8945.	408296	10.0%	[116037, 97684, 122294, 56152, 86260, 115737, 75932, 19887, 74589, 65451]
Cust. 8950.	1105517	10.0%	[12818, 92761, 69647, 78574, 1179, 48291, 115862, 18747, 54586, 45545]
Cust. 8951.	1383957	10.0%	[99523, 10386, 24901, 61195, 110090, 63855, 60180, 123579, 39918, 62040]
Cust. 8961.	1796894	20.0%	[21057, 37538, 68790, 75539, 81107, 28241, 1920, 90572, 49192, 111296]
Cust. 8964.	1493538	20.0%	[27127, 32173, 123197, 35835, 54226, 21057, 74344, 9074, 41166, 61329]
Cust. 8965.	2137148	10.0

Cust. 9246.	949007	10.0%	[100654, 27398, 48329, 123619, 121884, 36708, 98050, 66185, 78236, 84342]
Cust. 9249.	683528	10.0%	[81136, 123762, 46895, 47230, 37182, 108929, 113172, 69837, 96549, 12036]
Cust. 9251.	1105517	10.0%	[12818, 92761, 69647, 78574, 1179, 48291, 115862, 18747, 54586, 45545]
Cust. 9253.	84123	20.0%	[117225, 55796, 70703, 34705, 89291, 103373, 102245, 12915, 83201, 2934]
Cust. 9255.	419911	20.0%	[15054, 16135, 87451, 113287, 4574, 11938, 112480, 109146, 95239, 71663]
Cust. 9257.	683528	10.0%	[81136, 123762, 46895, 47230, 37182, 108929, 113172, 69837, 96549, 12036]
Cust. 9262.	87990	20.0%	[44434, 114995, 28471, 77218, 116830, 114051, 32018, 6172, 69533, 123649]
Cust. 9265.	684146	30.0%	[96878, 53760, 54208, 53539, 41684, 10547, 53260, 27220, 75186, 90304]
Cust. 9268.	2025046	20.0%	[71141, 91764, 45641, 115147, 101725, 14919, 72123, 11633, 87659, 80590]
Cust. 9278.	1952548	20.0%	[79337, 17204, 50686, 97884, 109882, 78475, 20860, 105696, 37548, 69923]
Cust. 9285.	1493538

Cust. 9589.	101478	30.0%	[106888, 64163, 107239, 24096, 71663, 3403, 95239, 97619, 120974, 9460]
Cust. 9590.	101478	30.0%	[106888, 64163, 107239, 24096, 71663, 3403, 95239, 97619, 120974, 9460]
Cust. 9591.	780744	10.0%	[18627, 42790, 57481, 82746, 59084, 120143, 18042, 114122, 98751, 98626]
Cust. 9592.	987186	10.0%	[83500, 97920, 29681, 10647, 98081, 113343, 87861, 33770, 117244, 35245]
Cust. 9600.	1176969	10.0%	[23473, 12716, 105054, 118443, 33076, 108994, 50246, 66890, 81017, 35143]
Cust. 9602.	1105517	10.0%	[12818, 92761, 69647, 78574, 1179, 48291, 115862, 18747, 54586, 45545]
Cust. 9604.	780744	10.0%	[18627, 42790, 57481, 82746, 59084, 120143, 18042, 114122, 98751, 98626]
Cust. 9612.	1352126	10.0%	[86243, 91334, 38897, 87968, 32894, 30124, 28693, 18887, 79637, 70468]
Cust. 9613.	780744	10.0%	[18627, 42790, 57481, 82746, 59084, 120143, 18042, 114122, 98751, 98626]
Cust. 9618.	2210274	30.0%	[63469, 6654, 59735, 83919, 86338, 120828, 106721, 119167, 77174, 34825]
Cust. 9619.	949680	10

Cust. 9970.	602635	30.0%	[57415, 64207, 6935, 94261, 51447, 12703, 5855, 17232, 119761, 58223]
Cust. 9978.	602635	30.0%	[57415, 64207, 6935, 94261, 51447, 12703, 5855, 17232, 119761, 58223]
Cust. 9983.	1897342	20.0%	[80222, 21100, 99951, 107416, 28563, 34675, 104312, 35605, 49013, 16707]
Cust. 9985.	1380100	40.0%	[66069, 77697, 86493, 69302, 81674, 84077, 6701, 64356, 45652, 8598]
Cust. 9987.	987186	10.0%	[83500, 97920, 29681, 10647, 98081, 113343, 87861, 33770, 117244, 35245]
Cust. 9989.	1476307	20.0%	[119980, 104709, 49921, 99501, 97586, 4112, 58513, 11430, 63809, 49667]
Cust. 9993.	84123	20.0%	[117225, 55796, 70703, 34705, 89291, 103373, 102245, 12915, 83201, 2934]
Cust. 9995.	87990	20.0%	[44434, 114995, 28471, 77218, 116830, 114051, 32018, 6172, 69533, 123649]
Cust. 9996.	1581321	10.0%	[89310, 4348, 32312, 72932, 30425, 121363, 89767, 72300, 45711, 13384]
Cust. 9997.	1475211	20.0%	[79637, 75776, 45542, 65820, 44797, 32310, 121848, 86243, 7447, 1290]
Cust. 9998.	1475211	20.0%	[79637

## Maticová faktorizácia

https://github.com/SurhanZahid/Recommendation-System-Using-Matrix-Factorization/blob/master/Recommender%20System%20With%20Matrix%20Factorization%20.ipynb

In [ ]:
mat_csr.toarray()

In [ ]:
mat_csr.shape

In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(mat_csr)
matrix.shape

In [ ]:
#corr = np.corrcoef(matrix)
corr.shape